In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [2]:
%matplotlib nbagg

In [3]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from matplotlib import cm
import libertem
from libertem import api

In [4]:
ctx = api.Context()

In [5]:
ds = ctx.load(
    "empad",
    path="/Users/weber/cachedata/acquisition_12.xml",
)
(scan_y, scan_x, detector_y, detector_x) = ds.shape
mask_shape = np.array((detector_y, detector_x))
cx = detector_x/2
cy = detector_y/2
y = scan_y//2
x = scan_x//2

DataSetException: could not initialize EMPAD file; error: [Errno 2] No such file or directory: '/Users/weber/cachedata/acquisition_12.xml'

In [ ]:
pick_analysis = ctx.create_pick_analysis(dataset=ds, x=x, y=y)

In [ ]:
%time pick_result = ctx.run(pick_analysis)

In [ ]:
print(pick_result)

In [ ]:
print(pick_result.intensity)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(pick_result.intensity.raw_data)

In [ ]:
def all_ones():
    return np.ones((detector_y, detector_x))

In [ ]:
def single_pixel():
    buf = np.zeros((detector_y, detector_x))
    buf[int(cy), int(cx)] = 1
    return buf

Here we use a mask UDF. This UDF uses the LiberTEM computation rather directly and returns a simple numerical result. See further below for examples that use the high-level analysis interface instead.

In [ ]:
mask_udf = libertem.udf.masks.ApplyMasksUDF(mask_factories=[all_ones, single_pixel])

In [ ]:
%%time
mask_udf_result = ctx.run_udf(dataset=ds, udf=mask_udf)

In [ ]:
print(mask_udf_result)

In [ ]:
print(mask_udf_result,intensity)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(mask_udf_result.intensity[..., 0], cmap=cm.gist_earth)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(mask_udf_result.intensity[..., 0], cmap=cm.gist_earth)

In [ ]:
mask_analysis = ctx.create_mask_analysis(dataset=ds, factories=[all_ones, single_pixel])

In [ ]:
%time mask_analysis_result = ctx.run(mask_analysis)

In [ ]:
print(mask_analysis_result)

In [ ]:
print(mask_analysis_result[0])

In [ ]:
fig, axes = plt.subplots()
axes.imshow(mask_analysis_result[0].visualized)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(mask_analysis_result[1].visualized)

In [ ]:
ro = min(detector_x,detector_y)/2
haadf_analysis = ctx.create_ring_analysis(dataset=ds, cx=cx, cy=cy, ro=ro, ri=ro*0.8)

In [ ]:
%time haadf_result = ctx.run(haadf_analysis)

In [ ]:
print(haadf_result)

In [ ]:
print(haadf_result.intensity)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(haadf_result.intensity.visualized)

In [ ]:
bf_analysis = ctx.create_disk_analysis(dataset=ds, cx=cx, cy=cy, r=ro*0.3)

In [ ]:
%time bf_result = ctx.run(bf_analysis)

In [ ]:
print(bf_result)

In [ ]:
print(bf_result.intensity)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(bf_result.intensity.visualized)

In [ ]:
point_analysis = ctx.create_point_analysis(dataset=ds, x=cx, y=cy)

In [ ]:
%time point_result = ctx.run(point_analysis)

In [ ]:
print(point_result)

In [ ]:
print(point_result.intensity)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(point_result.intensity.visualized)

In [ ]:
sum_analysis = ctx.create_sum_analysis(dataset=ds)

In [ ]:
%time sum_result = ctx.run(sum_analysis)

In [ ]:
print(sum_result)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(sum_result.intensity.raw_data)